In [1]:
import ast
import os
import re
import time


import pandas as pd
import numpy as np
import seaborn as sns


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

import warnings

warnings.simplefilter("ignore", DeprecationWarning)

from pyLDAvis import sklearn as sklearn_lda

import pyLDAvis


pd.options.display.max_colwidth = 280

In [2]:
table = pd.read_csv("resources/table_clean_topjuly16.csv", index_col = 0)

table.head()

,tweet_id,date,full_text,user,entities,short_url,number_urls,hashtags,user_mentions,media_url,full_text_processed
0,1283356992803868672,2020-07-15 11:05:25,le patron du pétrole de schiste dit que les etats unis ont passé leur pic traduction ici,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/w5MuRmDNnN', 'expanded_url': 'https://www.facebook.com/jeanmarc.jancovici/posts/10159506535177281', 'display_url': 'facebook.com/jeanmarc.janco…', 'indices': [92, 115]}, {'url': 'https://t.co/...","['https://t.co/w5MuRmDNnN', 'https://t.co/kx6WIwPNSs']",2,[],[],no_media,patron pétrole schiste etats unis pic traduction ici
1,1282394849870852098,2020-07-12 19:22:12,six mois sur une planète en crise carnet de voyage de greta thunberg des états unis à davos,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/DEskLJXWqU', 'expanded_url': 'https://medium.com/@jmj.fanpage/six-mois-sur-une-plan%C3%A8te-en-crise-carnet-de-voyage-de-greta-thunberg-des-%C3%A9tats-unis-%C3%A0-davos-629d92cef9cd', 'display...",['https://t.co/DEskLJXWqU'],1,[],[],no_media,mois planète crise carnet voyage greta thunberg états unis davos
2,1280935871471144962,2020-07-08 18:44:45,matthieu auzanneau de the shift project à propos du pic pétrolier en europe d ici 2030 si on sort du pétrole pour le climat nous sommes sauvés de mad max,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/lAC3hwH4o2', 'expanded_url': 'https://www.goodplanet.info/2020/07/08/matthieu-auzanneau-de-the-shift-project-a-propos-du-pic-petrolier-en-europe-dici-2030-si-on-sort-du-petrole-pour-le-climat-...",['https://t.co/lAC3hwH4o2'],1,[],[],no_media,matthieu auzanneau the shift project propos pic pétrolier europe d ici 2030 sort pétrole climat sommes sauvés mad max
3,1280850548518981633,2020-07-08 13:05:42,interview 3 50 jean marc jancovici revient sur la question climatique et de la croissance pour lui nous ne nous occupons pas du problème climatique alors qu il menace des milliards de personnes et l économie mondiale,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/YBmq94THuR', 'expanded_url': 'https://www.youtube.com/watch?v=ZsMSF0l9kyM', 'display_url': 'youtube.com/watch?v=ZsMSF0…', 'indices': [224, 247]}]}",['https://t.co/YBmq94THuR'],1,[],[],no_media,interview 3 50 jean marc jancovici revient question climatique croissance occupons problème climatique qu menace milliards personnes l économie mondiale
4,1280180257669238795,2020-07-06 16:42:12,vers un plan de transformation de l économie en faveur du climat et de la résilience the shift project vidéo 5 50,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/TWYTNdhx7t', 'expanded_url': 'https://www.youtube.com/watch?v=3sePULu7YIs&feature=youtu.be', 'display_url': 'youtube.com/watch?v=3sePUL…', 'indices': [118, 141]}]}",['https://t.co/TWYTNdhx7t'],1,[],[],no_media,plan transformation l économie faveur climat résilience the shift project vidéo 5 50


In [3]:
table["full_text_processed"] = table["full_text_processed"].values.astype("U")

In [4]:
# Inspecter les tweets de moins de 20 mots

In [5]:
table["len_text"] = table["full_text_processed"].map(
    lambda x: len(x)
)

In [6]:
table[table["len_text"] <= 20]

,tweet_id,date,full_text,user,entities,short_url,number_urls,hashtags,user_mentions,media_url,full_text_processed,len_text
22,1272156260260032520,2020-06-14 13:17:42,bon dimanche,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1272156218061185031, 'id_str': '1272156218061185031', 'indices': [17, 40], 'media_url': 'http://pbs.twimg.com/media/EaebVxRXYAcHeSl.jpg', 'media_url_https': 'https://pbs.twimg.com/media/EaebVxRX...",[],1,[],[],['https://t.co/5DUhapoiIG'],bon dimanche,12
159,1226489238042091521,2020-02-09 12:53:16,NaN,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/Af6wgi4YAG', 'expanded_url': 'https://www.linkedin.com/posts/jean-marc-jancovici_eni-est-ce-que-lon-appelle-un-p%C3%A9trolier-activity-6632216920605237248-3gyD', 'display_url': 'linkedin.com/p...",['https://t.co/Af6wgi4YAG'],1,[],[],no_media,nan,3
169,1226240910532521984,2020-02-08 20:26:30,NaN,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/28vS5oMNA9', 'expanded_url': 'https://www.linkedin.com/posts/jean-marc-jancovici_le-directeur-de-centrale-nantes-doit-%C3%AAtre-activity-6631440454661615616-Gkq1', 'display_url': 'linkedin.com...",['https://t.co/28vS5oMNA9'],1,[],[],no_media,nan,3
184,1218291438502522885,2020-01-17 21:58:08,et si les scientifiques avaient sous évalué le,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/NrEvnde8fX', 'expanded_url': 'https://www.20minutes.fr/planete/2697395-20200117-changement-climatique-selon-nouvelles-etudes-hausse-temperatures-pourrait-etre-plus-importante-prevu?fbclid=IwAR...",['https://t.co/NrEvnde8fX'],1,[],[],no_media,scientifiques évalué,20
197,1215071608840298498,2020-01-09 00:43:41,NaN,JMJancovici,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/87aUdyrPMS', 'expanded_url': 'https://www.facebook.com/jeanmarc.jancovici/posts/10158888602852281', 'display_url': 'facebook.com/jeanmarc.janco…', 'indices': [0, 23]}]}",['https://t.co/87aUdyrPMS'],1,[],[],no_media,nan,3
...,...,...,...,...,...,...,...,...,...,...,...,...
159,1141339439752536064,2019-06-19 13:38:22,NaN,OIL_MEN,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/plZ9jgxloA', 'expanded_url': 'https://www.google.com/amp/s/mobile.reuters.com/article/amp/idUSKCN1RO0DR', 'display_url': 'google.com/amp/s/mobile.r…', 'indices': [0, 23]}]}",['https://t.co/plZ9jgxloA'],1,[],[],no_media,nan,3
161,1141331577156317184,2019-06-19 13:07:07,body language,OIL_MEN,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/umqrhveLNF', 'expanded_url': 'https://twitter.com/IEABirol/status/1011749682224214016', 'display_url': 'twitter.com/IEABirol/statu…', 'indices': [17, 40]}]}",['https://t.co/umqrhveLNF'],1,[],[],no_media,body language,13
164,1140852108557803520,2019-06-18 05:21:53,NaN,OIL_MEN,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/yjLAMDX00p', 'expanded_url': 'https://ourfiniteworld.com/2019/06/12/seven-reasons-why-we-should-not-depend-on-imported-goods-from-china/', 'display_url': 'ourfiniteworld.com/2019/06/12/sev…', ...",['https://t.co/yjLAMDX00p'],1,[],[],no_media,nan,3
186,1139462154309029888,2019-06-14 09:18:42,pour mémoire encore,OIL_MEN,"{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/S8WETsEo0O', 'expanded_url': 'https://www.youtube.com/watch?v=wJ09U_a80MM&feature=youtu.be', 'display_url': 'youtube.com/watch?v=wJ09U_…', 'indices': [23, 46]}]}",['https://t.co/S8WETsEo0O'],1,[],[],no_media,mémoire,7


In [17]:
less_20 = " ".join(t for t in table[table["len_text"] <= 20]["full_text_processed"].astype(str).values.tolist())
less_20

'bon dimanche nan nan scientifiques évalué nan situation critique nan nan rejeterons l ouragan article complet ouragan dorian ceta controversé planète fournaise texte commentaire complet commentaire lire ici nan remettait ipbes alimentation carte face risque co2 pourrez émettre lettre l éléphant   édito vidéo états grande accélération pollué   surprise   infographie haut conseil climat climat   soleil   soleil   climat bce nan club rome climat avenir gaz france basculer menace biodiversité diplomatie appel blocages matthieu nan omg my gallon of oil climat cause perdue   interrogations l europe fois n coutume nan nan j arrive connecter test monde carbone 2050     nan nan pdg total nan nan   nan nan nan nan nan nan nan belle journée 300 mds sûr france vraie consternant fred vargas rt mpdelagontrie allez rire nan jamais nan l honneur l europe robert nan nan ï muchas gracias nan jean indeed xlanata indeed vraie victoire alas relire rater prétexte   mcvulcain oui   juliensergent3   fab75017

In [18]:
from nltk import FreqDist
all_words = less_20.split() # list of all the words in your corpus
fdist = FreqDist(all_words) # a frequency distribution of words (word count over the corpus)
k = 10000 # say you want to see the top 10,000 words
top_k_words, _ = zip(*fdist.most_common(k)) # unzip the words and word count tuples
print(top_k_words)

('nan', 'l', 'oui', 'climat', 'j', 'rt', 'c', 'n', 'bensabaugcap', 'france', 'sûr', 'orhekto', 's', 'gaurelmaaron', 'x', 'bon', 'ouragan', 'complet', 'commentaire', 'soleil', 'europe', 'fois', 'vraie', 'allez', 'indeed', 'moukiserge', 'waxstorm_fr', 'parisdix', 'lien', 'dites', 'pascalout69', 'd', 'astrochnis', 'échange', 'ildrugo', '_', '4', 'ericvidalenc', 'vite', 'plou', 'it', 'dis', 'dimanche', 'scientifiques', 'évalué', 'situation', 'critique', 'rejeterons', 'article', 'dorian', 'ceta', 'controversé', 'planète', 'fournaise', 'texte', 'lire', 'ici', 'remettait', 'ipbes', 'alimentation', 'carte', 'face', 'risque', 'co2', 'pourrez', 'émettre', 'lettre', 'éléphant', 'édito', 'vidéo', 'états', 'grande', 'accélération', 'pollué', 'surprise', 'infographie', 'haut', 'conseil', 'bce', 'club', 'rome', 'avenir', 'gaz', 'basculer', 'menace', 'biodiversité', 'diplomatie', 'appel', 'blocages', 'matthieu', 'omg', 'my', 'gallon', 'of', 'oil', 'cause', 'perdue', 'interrogations', 'coutume', 'arriv

/home/jeanberyl/anaconda3/envs/pythonNLP/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/home/jeanberyl/anaconda3/envs/pythonNLP/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, defaultdict
/home/jeanberyl/anaconda3/envs/pythonNLP/lib/python3.7/site-packages/nltk/lm/vocabulary.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Counter, Iterable


In [ ]:
###### Frequency ######

In [ ]:
from nltk import FreqDist
all_words = text.split() # list of all the words in your corpus
fdist = FreqDist(all_words) # a frequency distribution of words (word count over the corpus)
k = 10000 # say you want to see the top 10,000 words
top_k_words, _ = zip(*fdist.most_common(k)) # unzip the words and word count tuples
print(top_k_words)

In [4]:
####### Vectorizing #######

In [21]:
count_vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words = ['rt', 'qu', 'to', 'of', 'in', 'the', 'is', 'and',
                                                                     'faire', 'by', '12', 'retenu', '20', 'gt'])
count_data = count_vectorizer.fit_transform(table[table["len_text"] >= 20]["full_text_processed"])

In [24]:
# Tweak the two parameters below
number_topics = 6
number_words = 200# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1, verbose = 0)
lda.fit(count_data)# Print the topics found by the LDA model

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=6, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [25]:
%%time
# Visualize the topics
pyLDAvis.enable_notebook()



LDAvis_prepared = sklearn_lda.prepare(lda, count_data, count_vectorizer)
        


LDAvis_prepared

CPU times: user 1min 42s, sys: 488 ms, total: 1min 42s
Wall time: 1min 51s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.088951  0.099564       1        1  19.070388
5      0.091355  0.026483       2        1  17.549921
1      0.069024  0.035131       3        1  17.137959
0     -0.029451 -0.067709       4        1  16.048736
4     -0.008577 -0.044471       5        1  15.194968
2     -0.033400 -0.048999       6        1  14.998028, topic_info=                       Term        Freq       Total Category  logprob  loglift
94659             jean marc  248.000000  248.000000  Default  30.0000  30.0000
104023       marc jancovici  244.000000  244.000000  Default  29.0000  29.0000
94662   jean marc jancovici  243.000000  243.000000  Default  28.0000  28.0000
104014                 marc  251.000000  251.000000  Default  27.0000  27.0000
94596                  jean  294.000000  294.000000  Default  26.0000  26.0000
...                     ...         ...         ...      ...      ...      ...
186359          énergétique   25.072040  188.054363   Topic6  -7.5377  -0.1177
183948           électrique   22.454872  120.045487   Topic6  -7.6479   0.2209
132937                 prix   22.876464  173.525454   Topic6  -7.6293  -0.1290
45721                 crise   20.517952  110.013535   Topic6  -7.7382   0.2179
31257            changement   20.532408  138.296007   Topic6  -7.7374  -0.0101

[493 rows x 6 columns], token_table=        Topic      Freq                  Term
term                                         
397         1  0.133008                   100
397         2  0.166260                   100
397         3  0.133008                   100
397         4  0.116382                   100
397         5  0.315894                   100
...       ...       ...                   ...
186359      3  0.159528           énergétique
186359      4  0.079764           énergétique
186359      5  0.202069           énergétique
186359      6  0.132940           énergétique
187830      5  0.988240  états unis émissions

[971 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 2, 1, 5, 3])

In [23]:
%%time
# Visualize the topics
pyLDAvis.enable_notebook()



LDAvis_prepared = sklearn_lda.prepare(lda, count_data, count_vectorizer)
        


LDAvis_prepared

CPU times: user 1min 40s, sys: 440 ms, total: 1min 40s
Wall time: 1min 54s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.156860 -0.000574       1        1  11.347735
6     -0.024687  0.080878       2        1  10.596407
7     -0.044595 -0.001316       3        1  10.469690
9     -0.020928 -0.146177       4        1  10.447435
0     -0.044368 -0.017177       5        1  10.130382
5      0.002466  0.013104       6        1  10.056610
8     -0.024829  0.027466       7        1  10.049543
1      0.002134  0.010604       8        1   9.764280
4     -0.036881  0.032386       9        1   8.596593
3      0.034829  0.000806      10        1   8.541326, topic_info=                       Term        Freq       Total Category  logprob  loglift
104023       marc jancovici  248.000000  248.000000  Default  30.0000  30.0000
94662   jean marc jancovici  247.000000  247.000000  Default  29.0000  29.0000
94659             jean marc  252.000000  252.000000  Default  28.0000  28.0000
104014                 marc  255.000000  255.000000  Default  27.0000  27.0000
93705             jancovici  274.000000  274.000000  Default  26.0000  26.0000
...                     ...         ...         ...      ...      ...      ...
46189            croissance   14.256505  122.529234  Topic10  -7.5392   0.3091
124535                 pays   14.948037  201.743495  Topic10  -7.4919  -0.1422
185095            émissions   14.469537  212.237229  Topic10  -7.5244  -0.2254
110030                monde   14.208130  214.102920  Topic10  -7.5426  -0.2524
36787                   co2   14.022586  178.254016  Topic10  -7.5558  -0.0823

[814 rows x 6 columns], token_table=        Topic      Freq                 Term
term                                        
316         7  0.800801         10 dernières
317         7  0.800801  10 dernières années
624         1  0.852045              11 2018
1284        8  0.969236                  173
1359        2  0.789095               18 mai
...       ...       ...                  ...
187956      3  0.594715             étranger
187956      5  0.198238             étranger
187956      6  0.198238             étranger
188333      7  0.802082               évalue
188333      8  0.267361               évalue

[2080 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 8, 10, 1, 6, 9, 2, 5, 4])